## **FOUNDATION PROJECT - GROUP ASSIGNMENT** ##

> **Use Case ::** Predicting Stock Movement - **"Stock Market Copilot"**

> **Dataset Source ::** Yahoo Finance - https://finance.yahoo.com/quote/TSLA/history/?filter=history

> **Group No. ::** 6

## **USER INTERFACE - STREAMLIT**

In [1]:
pip install streamlit altair uvicorn pyngrok --no-warn-script-location

In [19]:
# Importing necessary libraries
from multiprocessing import Process
from threading import Thread
from pyngrok import ngrok
import streamlit as st
import pandas as pd
import numpy as np
from joblib import load
import os
import plotly.graph_objects as go

import seaborn as sn
import matplotlib.pyplot as plt

# Google Colab Notbook related imports
from google.colab import userdata

### **VII.  User Interface - Streamlit**
> In this stage we will creating a User interface for user to select the input values and display the predicted value through the best performance model deployed. The UI app will use the FastAPI service hosted at localhost:8501 to post the post the request to model and fetch the value.

> **Streamlit -**
>> Streamlit is a free and open-source framework to rapidly build and share web apps without extensive web development knowledge.

#### **Creating the APP**
> We will be defining the app layout i.e., the fields that are to be displayed to the user over the screen and the type of field to be provided (e.g., dropdown for stratified data or text field etc.) along the set of values acceptable in each field (e.g, range of values for free text or definite values to be displayed in dropdown).

**Steps Performed -**
1. We create a python file app.py that contains the outline of the webpage that is to be generated via streamlit library. For this we define the page title name, header, parameters that are to be displayed on the page and whose values are to be captured and posted to once entered by user on the web page.
2. For each parameter defined, we provide the type of field (slider, free text box etc.), data type and range of values to be accepted.
3. We then define the payload format that is to be posted when the "Predict" button is clicked.
4. The web application is hosted at a localserver started at endpoint 0.0.0.0:8501 using the uvicorn module. The streamlit module automatically converts the outline defined in app.py to html content with a user friendly interface.
5. When user enters the details and clicks Predict the request is received by the app and posted to the secure API endpoint exposed over public URL.

**Note - Pls ensure the "ngrokPublicURL.txt" file to be fetched and uploaded from Notebook-2 to the local storage of this notebook.**

**Using NGROK::**

- We use ngrok to create a secure endpoint to which an external user can access the web application hosted. Ngrok creates a secure tunnel between the public exposed endpoint and the local server (0.0.0.0:8501) end point on which the service is running.
- Once the enters the details on the page and hits the Predict button,traffic/request hits the public endpoint, ngrok forwards the traffic over the secure channel thereby abstracting the internal working endpoint from outside world.


In [3]:
pip install scikit-learn==1.3.2

In [15]:
%%writefile app.py

from pyngrok import ngrok
import streamlit as st
import pandas as pd
import numpy as np
from joblib import load
import requests
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import plotly.graph_objects as go

# Load FastAPI public URL
with open("ngrokPublicURL.txt", "r") as f:
    endPointUrl = f.read().strip()

def create_streamlit_app():
    """Streamlit app with prediction visualization"""
    st.set_page_config(page_title="Stock Market Copilot", layout="wide")

    # Header
    st.title("Stock Market Copilot")

    st.markdown("""
    <style>
    /* General app style */
    .stAlert { padding: 20px; }

    /* Text improvements */
    .st-b7 {
        color: black !important;
    }

    /* Background improvements */
    .st-cg, .stSelectbox div[data-baseweb="select"] {
        background-color: white !important;
        color: black !important;
    }

    /* Dropdown styling */
    div[data-baseweb="select"] > div {
        background-color: white !important;
        color: black !important;
        border-radius: 5px;
        padding: 1px;
        font-size: 16px;
    }

    /* Menu items */
    div[data-baseweb="menu"] {
        background-color: white !important;
        color: black !important;
    }

    div[data-baseweb="menu"] > div {
        background-color: white !important;
        color: black !important;
    }

    /* Hover effect */
    div[data-baseweb="option"]:hover {
        background-color: #f0f0f0 !important;
        color: black !important;
    }
    </style>
    """, unsafe_allow_html=True)

    stock_name = st.selectbox(
        "Select Prediction Period",
        ["APPLE (AAPL)", "Tesla, Inc. (TSLA)", "Berkshire Hathaway Inc. (BRK-B)"],
        index=0
    )

    # User inputs
    prediction_date = st.date_input(
        "Prediction Start Date",
        value=datetime.now(),
        min_value=datetime.now(),
        max_value=datetime.now() + timedelta(days=365)
    )

    prediction_period = st.selectbox(
        "Select Prediction Period",
        ["2 weeks", "1 month", "3 months"],
        index=0
    )

    # Submit button
    if st.button("Generate Predictions"):
        with st.spinner("Generating predictions..."):
            try:
                n_days = 10 if prediction_period == "2 weeks" else \
                         20 if prediction_period == "1 month" else \
                         60 if prediction_period == "3 months" else 90

                response = requests.post(
                    endPointUrl + "/predict",
                    json={
                        "Date": prediction_date.strftime('%Y-%m-%d'),
                        "n_days": n_days,
                        "range": prediction_period,
                        "stock": stock_name
                    },
                    headers={
                        "Content-Type": "application/json"
                    },
                    timeout=30
                )

                if response.status_code != 200:
                    raise Exception(f"API Error: {response.text}")

                results = response.json()
                predictions = pd.DataFrame(results['predictions'])
                validation = pd.DataFrame(results['validation'])

                predictions['Date'] = pd.to_datetime(predictions['Date'])
                predictions['Day'] = predictions['Date'].dt.day_name()
                predictions['Week'] = predictions['Date'].dt.isocalendar().week

                if not validation.empty:
                    validation['Date'] = pd.to_datetime(validation['Date'])

                # Plotting
                fig = go.Figure()

                if not validation.empty:
                    fig.add_trace(go.Scatter(
                        x=validation['Date'],
                        y=validation['Close'],
                        name='Historical Data',
                        line=dict(color='blue', width=2),
                        hovertemplate='Date: %{x|%Y-%m-%d}<br>Value: %{y:.2f}<extra></extra>'
                    ))

                predictions['Change'] = predictions['Predicted'].diff()
                predictions['Direction'] = predictions['Change'].apply(
                    lambda x: 'green' if x > 0 else ('red' if x < 0 else 'gray')
                )

                predictions['Prev_Date'] = predictions['Date'].shift(1).dt.strftime('%Y-%m-%d')
                predictions['Prev_Day'] = predictions['Day'].shift(1)
                predictions['Prev_Week'] = predictions['Week'].shift(1)
                predictions['Prev_Value'] = predictions['Predicted'].shift(1)

                for i in range(1, len(predictions)):
                    fig.add_trace(go.Scatter(
                        x=predictions['Date'].iloc[i-1:i+1],
                        y=predictions['Predicted'].iloc[i-1:i+1],
                        line=dict(
                            color=predictions['Direction'].iloc[i],
                            width=2
                        ),
                        mode='lines+markers',
                        marker=dict(size=8, color=predictions['Direction'].iloc[i-1]),
                        showlegend=False,
                        customdata=np.array([[
                            predictions['Day'].iloc[i],
                            int(predictions['Week'].iloc[i]),
                            predictions['Prev_Date'].iloc[i],
                            predictions['Prev_Day'].iloc[i],
                            int(predictions['Prev_Week'].iloc[i]),
                            float(predictions['Prev_Value'].iloc[i-1])
                        ]]),
                        hovertemplate=(
                            '<b>Current Day</b><br>'
                            'Date: %{x|%Y-%m-%d}<br>'
                            'Day: %{customdata[0]}<br>'
                            'Week: %{customdata[1]}<br>'
                            'Value: %{y:.2f}<br><br>'
                            '<b>Previous Day</b><br>'
                            'Date: %{customdata[2]}<br>'
                            'Day: %{customdata[3]}<br>'
                            'Week: %{customdata[4]}<br>'
                            'Value: %{customdata[5]:.2f}'
                            '<extra></extra>'
                        )
                    ))

                fig.update_layout(
                    title='Stock Price Prediction with Directional Trends',
                    xaxis_title='Date',
                    yaxis_title='Price ($)',
                    hovermode='closest',
                    xaxis=dict(
                        range=[
                            predictions['Date'].min() - timedelta(days=3),
                            predictions['Date'].max() + timedelta(days=3)
                        ]
                    ),
                    legend=dict(
                        orientation="h",
                        yanchor="bottom",
                        y=1.02,
                        xanchor="right",
                        x=1
                    )
                )

                st.plotly_chart(fig, use_container_width=True)

                # Detailed prediction table
                with st.expander("View Detailed Prediction Data"):
                    def style_predictions(row):
                        if row.name == 0:
                            return [''] * len(row)
                        return ['background-color: lightgreen' if row['Predicted'] > predictions.at[row.name-1, 'Predicted']
                                else 'background-color: lightcoral' for _ in row]

                    st.dataframe(predictions.style.apply(style_predictions, axis=1))

                # Historical validation
                if not validation.empty:
                    with st.expander("View Historical Validation Data"):
                        st.dataframe(validation)

            except Exception as e:
                st.error(f"Prediction failed: {str(e)}")

    # Footer disclaimer
    st.markdown(f"""
    ---
    **Disclaimer:** This application is for informational purposes only and should not be considered as financial advice.
    The predictions are based on historical data and statistical models, and past performance is not indicative of future results.
    Always conduct your own research and consult with a qualified financial advisor before making investment decisions.

    Data Source: Yahoo Finance | Model Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
    """)

if __name__ == "__main__":
    create_streamlit_app()


Overwriting app.py


In [20]:
%%writefile app.py

from pyngrok import ngrok
import streamlit as st
import pandas as pd
import numpy as np
from joblib import load
import requests
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sn
import matplotlib.pyplot as plt

# Load FastAPI public URL for Prediction
with open("ngrokPublicURL.txt", "r") as f:
    endPointUrl = f.read().strip()

# Load FastAPI public URL for Drift Check
with open("ngrokPublicURL2.txt", "r") as g:
    endPointUrl2 = g.read().strip()

def create_streamlit_app():
    """Streamlit app with prediction and drift visualization"""
    st.set_page_config(page_title="Stock Market Copilot", layout="wide")

    # Header
    st.title("Stock Market Copilot")

    # Custom Style
    st.markdown("""
    <style>
    .stAlert { padding: 20px; }
    .st-b7 { color: black !important; }
    .st-cg, .stSelectbox div[data-baseweb="select"],
    div[data-baseweb="select"] > div,
    div[data-baseweb="menu"],
    div[data-baseweb="menu"] > div {
        background-color: white !important;
        color: black !important;
        border-radius: 5px;
        padding: 1px;
        font-size: 16px;
    }
    div[data-baseweb="option"]:hover {
        background-color: #f0f0f0 !important;
        color: black !important;
    }
    </style>
    """, unsafe_allow_html=True)

    # User Inputs
    stock_name = st.selectbox(
        "Select Stock",
        ["APPLE (AAPL)", "Tesla, Inc. (TSLA)", "Berkshire Hathaway Inc. (BRK-B)"],
        index=0
    )

    prediction_date = st.date_input(
        "Prediction Start Date",
        value=datetime.now(),
        min_value=datetime.now(),
        max_value=datetime.now() + timedelta(days=365)
    )

    prediction_period = st.selectbox(
        "Select Prediction Period",
        ["2 weeks", "1 month", "3 months"],
        index=0
    )

    # Column for Buttons
    col1, col2 = st.columns([1, 1])

    with col1:
        if st.button("Generate Predictions"):
            with st.spinner("Generating predictions..."):
                try:
                    n_days = 10 if prediction_period == "2 weeks" else \
                             20 if prediction_period == "1 month" else 60

                    response = requests.post(
                        endPointUrl + "/predict",
                        json={
                            "Date": prediction_date.strftime('%Y-%m-%d'),
                            "n_days": n_days,
                            "range": prediction_period,
                            "stock": stock_name
                        },
                        headers={"Content-Type": "application/json"},
                        timeout=30
                    )

                    if response.status_code != 200:
                        raise Exception(f"API Error: {response.text}")

                    results = response.json()
                    predictions = pd.DataFrame(results['predictions'])
                    validation = pd.DataFrame(results['validation'])

                    predictions['Date'] = pd.to_datetime(predictions['Date'])
                    predictions['Day'] = predictions['Date'].dt.day_name()
                    predictions['Week'] = predictions['Date'].dt.isocalendar().week

                    if not validation.empty:
                        validation['Date'] = pd.to_datetime(validation['Date'])

                    # Plotting logic
                    fig = go.Figure()

                    if not validation.empty:
                        fig.add_trace(go.Scatter(
                            x=validation['Date'],
                            y=validation['Close'],
                            name='Historical Data',
                            line=dict(color='blue', width=2),
                            hovertemplate='Date: %{x|%Y-%m-%d}<br>Value: %{y:.2f}<extra></extra>'
                        ))

                    predictions['Change'] = predictions['Predicted'].diff()
                    predictions['Direction'] = predictions['Change'].apply(
                        lambda x: 'green' if x > 0 else 'red'
                    )
                    predictions['Prev_Date'] = predictions['Date'].shift(1).dt.strftime('%Y-%m-%d')
                    predictions['Prev_Day'] = predictions['Day'].shift(1)
                    predictions['Prev_Week'] = predictions['Week'].shift(1)
                    predictions['Prev_Value'] = predictions['Predicted'].shift(1)

                    for i in range(1, len(predictions)):
                        fig.add_trace(go.Scatter(
                            x=predictions['Date'].iloc[i-1:i+1],
                            y=predictions['Predicted'].iloc[i-1:i+1],
                            line=dict(
                                color=predictions['Direction'].iloc[i],
                                width=2
                            ),
                            mode='lines+markers',
                            marker=dict(size=8, color=predictions['Direction'].iloc[i]),
                            showlegend=False,
                            customdata=np.array([[  # Custom tooltip info
                                predictions['Day'].iloc[i],
                                int(predictions['Week'].iloc[i]),
                                predictions['Prev_Date'].iloc[i],
                                predictions['Prev_Day'].iloc[i],
                                int(predictions['Prev_Week'].iloc[i]),
                                float(predictions['Prev_Value'].iloc[i])
                            ]]),
                            hovertemplate=(
                                '<b>Current Day</b><br>'
                                'Date: %{x|%Y-%m-%d}<br>'
                                'Day: %{customdata[0]}<br>'
                                'Week: %{customdata[1]}<br>'
                                'Value: %{y:.2f}<br><br>'
                                '<b>Previous Day</b><br>'
                                'Date: %{customdata[2]}<br>'
                                'Day: %{customdata[3]}<br>'
                                'Week: %{customdata[4]}<br>'
                                'Value: %{customdata[5]:.2f}'
                                '<extra></extra>'
                            )
                        ))

                    fig.update_layout(
                        title='Stock Price Prediction with Directional Trends',
                        xaxis_title='Date',
                        yaxis_title='Price ($)',
                        hovermode='closest',
                        xaxis=dict(
                            range=[
                                predictions['Date'].min() - timedelta(days=3),
                                predictions['Date'].max() + timedelta(days=3)
                            ]
                        ),
                        legend=dict(
                            orientation="h",
                            yanchor="bottom",
                            y=1.02,
                            xanchor="right",
                            x=1
                        )
                    )
                    st.plotly_chart(fig, use_container_width=True)

                    with st.expander("View Detailed Prediction Data"):
                        def style_predictions(row):
                            if row.name == 0:
                                return [''] * len(row)
                            return ['background-color: lightgreen' if row['Predicted'] > predictions.at[row.name-1, 'Predicted']
                                    else 'background-color: lightcoral' for _ in row]
                        st.dataframe(predictions.style.apply(style_predictions, axis=1))

                    if not validation.empty:
                        with st.expander("View Historical Validation Data"):
                            st.dataframe(validation)

                except Exception as e:
                    st.error(f"Prediction failed: {str(e)}")

    with col2:
        if st.button("Drift Check"):
            with st.spinner("Checking data drift..."):
                try:
                    n_days = 10 if prediction_period == "2 weeks" else \
                             20 if prediction_period == "1 month" else 60

                    response = requests.post(
                        endPointUrl2 + "/checkDrift",
                        json={
                            "Date": prediction_date.strftime('%Y-%m-%d'),
                            "n_days": n_days,
                            "range": prediction_period,
                            "stock": stock_name
                        },
                        headers={"Content-Type": "application/json"},
                        timeout=30
                    )

                    if response.status_code != 200:
                        raise Exception(f"Drift API Error: {response.text}")

                    result = response.json()

                    st.markdown(f"### Overall Drift Status: **{'Drift Detected' if result['overall_drift_status'] else 'No Significant Drift'}**")
                    st.markdown("### Feature-wise Drift Summary:")
                    summary_df = pd.DataFrame(result['feature_summary'])
                    st.dataframe(summary_df)

                    st.markdown("### KDE Plots of Numeric Features (Train vs Prediction)")

                    train_df = pd.DataFrame(result['numeric_features_train'])
                    prod_df = pd.DataFrame(result['numeric_feature_predict'])
                    num_vars = result['num_features']

                    for num_feature in num_vars:
                        fig, ax = plt.subplots(figsize=(8, 4))

                        sn.kdeplot(train_df[num_feature], alpha=0.3, color='purple', fill=True, label='Train Dataset', ax=ax)
                        sn.kdeplot(prod_df[num_feature], alpha=0.3, color='yellow', fill=True, label='Prediction Dataset', ax=ax)

                        ax.set_title(f"Distribution of Feature :: {num_feature}")
                        ax.set_xlabel(num_feature)
                        ax.set_ylabel("Density")
                        ax.legend(loc='upper right')

                        st.pyplot(fig)

                except Exception as e:
                    st.error(f"Drift check failed: {str(e)}")

    # Footer disclaimer
    st.markdown(f"""
    ---
    **Disclaimer:** This application is for informational purposes only and should not be considered as financial advice.
    The predictions are based on historical data and statistical models, and past performance is not indicative of future results.
    Always conduct your own research and consult with a qualified financial advisor before making investment decisions.

    Data Source: Yahoo Finance | Model Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
    """)

if __name__ == "__main__":
    create_streamlit_app()


Overwriting app.py


#### **Running the UI**
> The UI service shall be hosted on port 8501 in localhost.

In [5]:
# Storing the ngrok auth-token which will be later used to authorize the web user posting the API request when connecting to the API service hosted at port 8000
ngrok.set_auth_token("2oy6VhkcQYpcuGwVNoaRhKrA5T6_vwBUcQ4iShdyFHoMnCD4")

In [6]:
# Start the Streamlit server in a separate thread so that the execution of main programme running in this notebook is not interrupted
streamlit_thread = Thread(
    target=lambda: os.system("streamlit run app.py --server.port 8501"), daemon=True
)
streamlit_thread.start()

# Expose the Streamlit app through ngrok
# ngrokPublicURL.txt file to be fetched and uploaded from Notebook-2
streaming_url = ngrok.connect(8501)
print(f"Streamlit public URL: {streaming_url}")

Streamlit public URL: NgrokTunnel: "https://91e7-35-194-75-30.ngrok-free.app" -> "http://localhost:8501"
